In [1]:
########################
## A simple example of optimal alignment of lists, to maximize first/leading word alignment (from two fitted topic models).
##
## Author: Chris Meaney
## Date: December 2023
########################

In [2]:
## Dependencies
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import pandas as pd

In [3]:
## Define "first-word alignment" loss function
def match_first_word_loss(a, b):
    return int(a[0]==b[0])

In [4]:
## Define some function to optimize semantic agreement - over arbitrary lists of elements A and B
## The function takes as inputs two lists of lists: A=[[],[],[]] and B=[[],[],[]] for example from two topic models
## The function returns the optimal alignment of the two lists/matrices using first word agreement objective function
def optimize_alignment(A, B):
    model = gp.Model("list_alignment")

    # Variables
    alignment_vars = {}
    for i in range(len(A)):
        for j in range(len(B)):
            alignment_vars[i, j] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")

    # Objective function
    obj_expr = gp.LinExpr()
    for i in range(len(A)):
        for j in range(len(B)):
            obj_expr += alignment_vars[i, j] * match_first_word_loss(A[i], B[j])

    model.setObjective(obj_expr, GRB.MAXIMIZE)

    # Constraints
    for i in range(len(A)):
        model.addConstr(gp.quicksum(alignment_vars[i, j] for j in range(len(B))) == 1)

    for j in range(len(B)):
        model.addConstr(gp.quicksum(alignment_vars[i, j] for i in range(len(A))) == 1)

    # Solve the model
    model.optimize()

    # Extract the solution
    alignment = []
    for i in range(len(A)):
        for j in range(len(B)):
            if alignment_vars[i, j].x == 1:
                alignment.append((A[i], B[j]))

    return alignment

In [5]:
## Import pandas dataFrame corresponding to two topical matrices 
file_dir = "C:\\Users\\ChristopherMeaney\\Desktop\\tmp\\pyGurobi_LinearAssignmentExample\\"

utopian_path = file_dir + "FINAL_UTOPIAN_TopicTable_NMF_K=50.csv"
emrpc_path = file_dir + "FINAL_EMRPC_TopicTable_NMF_K=50.csv"

utopian = pd.read_csv(utopian_path)
emrpc = pd.read_csv(emrpc_path)

In [6]:
## UTOPIAN topic table
utopian.iloc[:,1:6]

,word1,word2,word3,word4,word5
0,tylenol,advil,tab,headache,tabs
1,mg,tab,tabs,capsules,po
2,fever,diarrhea,vomiting,tylenoladvil,viral
3,neck,head,arm,headache,headaches
4,bw,iron,tsh,ferritin,thyroid
5,work,social,stress,working,treatment
6,bp,systolic,diastolic,htn,norvasc
7,sleep,bed,sleeping,apnea,insomnia
8,anxiety,anxious,panic,social,counselling
9,flu,shot,anaphylactic,influenza,ibuprofen


In [7]:
## EMRPC topic table
emrpc.iloc[:,1:6]

,word1,word2,word3,word4,word5
0,pain,chronic,tylenol,gabapentin,percocet
1,ct,scan,head,wife,ultrasound
2,inr,dosage,mg,thx,wife
3,hip,tylenol,xray,oa,physio
4,insulin,dm,lantus,diabetes,fbs
5,flu,diet,exercise,ldl,medications
6,mg,tablet,qhs,tablets,hydromorphone
7,surgery,hospital,surgeon,eye,discharge
8,er,feeling,hospital,discharge,admitted
9,urine,uti,macrobid,culture,neg


In [8]:
## Convert each of the above dataFrames into a list of list data structure
utopian_list = utopian.iloc[:, 1:6].values.tolist()
emrpc_list = emrpc.iloc[:, 1:6].values.tolist()

[len(utopian_list), len(emrpc_list)]

[50, 50]

In [9]:
# Optimize alignment between two topical summary matrices
alignment_result = optimize_alignment(utopian_list, emrpc_list)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 2500 columns and 5000 nonzeros
Model fingerprint: 0x7e2a5598
Variable types: 0 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.05s
Presolved: 100 rows, 2500 columns, 5000 nonzeros
Variable types: 0 continuous, 2500 integer (2500 binary)

Root relaxation: objective 2.700000e+01, 120 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [10]:
# Print the result
# for a, b in alignment_result:
#    print(f"Align: {a} and {b}")
#    print("Match First Word Loss: ", int(a[0]==b[0]))
#    print('\n')

In [11]:
##
## Return the aligned lists as a pandas DataFrame
##
emrpc_list = []
utopian_list = []
intersection_list = []

# Print the result
for a, b in alignment_result:
    emrpc_list.append(b)
    utopian_list.append(a)
    intersection_list.append(a[0]==b[0])

res_df = pd.DataFrame({
    'emrpc': emrpc_list,
    'utopian': utopian_list,
    'intersection': intersection_list
})

res_df_ = res_df.sort_values('intersection', ascending=False)
res_df_

,emrpc,utopian,intersection
25,"[urine, uti, macrobid, culture, neg]","[urine, uti, urinary, dysuria, hematuria]",True
19,"[dose, bloods, msg, db, thx]","[dose, medication, immunization, injection, sh...",True
48,"[referral, ccac, mental, sw, msg]","[referral, derm, ent, gi, mri]",True
47,"[skin, rash, lesion, breast, lesions]","[skin, rash, cream, derm, lesions]",True
45,"[hip, tylenol, xray, oa, physio]","[hip, xray, oa, physio, flexion]",True
44,"[feels, feeling, feel, husband, worried]","[feels, felt, tired, stress, anxious]",True
41,"[blood, pressure, glucose, bleeding, diabetes]","[blood, pressure, medication, pulse, pounds]",True
40,"[knee, oa, swelling, xray, medial]","[knee, swelling, oa, joint, medial]",True
39,"[chest, clear, sob, heart, edema]","[chest, sob, cvs, edema, palpitations]",True
37,"[back, physio, spine, legs, lumbar]","[back, spine, lumbar, flexion, physio]",True


In [12]:
##
## Compute objective function
##
res_df_.intersection.sum()

27

In [13]:
## Note: for the 33 topical vectors where first words DO NOT match, the alignment/solution is arbitrary
## i.e. there exist several other solutions where one permutes the topical vectors without matching first word/token

In [14]:
##########################
## Write solution to disk
##########################

In [15]:
out_path = "C:\\Users\\ChristopherMeaney\\Desktop\\tmp\\pyGurobi_LinearAssignmentExample\\MILP_Solver_MatchFirstWordObjective.csv"
res_df_.to_csv(path_or_buf=out_path, sep=',', index=False)

In [16]:
#######################################
## Properties of noteboook
#######################################

In [17]:
## Notebook last run on following date
from datetime import date
print(date.today())

2023-12-27


In [18]:
## Session information
# !pip install session_info
import session_info
session_info.show()